## はじめに

このレッスンでは以下の内容を扱います：
- ファンクションコールとは何か、その利用例
- OpenAIを使ったファンクションコールの作成方法
- アプリケーションへのファンクションコールの組み込み方

## 学習目標

このレッスンを終えると、次のことができるようになります：

- ファンクションコールを使う目的を理解する
- OpenAIサービスを使ってファンクションコールを設定する
- アプリケーションのユースケースに合わせて効果的なファンクションコールを設計する


## 関数呼び出しの理解

このレッスンでは、教育系スタートアップのために、ユーザーがチャットボットを使って技術系コースを探せる機能を作成します。ユーザーのスキルレベル、現在の役割、興味のある技術に合ったコースをおすすめします。

これを実現するために、以下を組み合わせて使います：
 - `OpenAI` を使ってユーザー向けのチャット体験を作成
 - `Microsoft Learn Catalog API` を使って、ユーザーのリクエストに基づきコースを検索
 - `Function Calling` を使って、ユーザーの問い合わせを関数に渡し、APIリクエストを実行

まず、なぜ関数呼び出しを使うのかを見ていきましょう：

print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # GPTが関数のレスポンスを確認できるように新しいレスポンスを取得

print(second_response.choices[0].message)


### なぜFunction Callingが必要なのか

このコースの他のレッスンを終えたことがあれば、Large Language Models（LLMs）の強力さはすでにご存知でしょう。同時に、その限界についても感じているかもしれません。

Function Callingは、OpenAI Serviceの機能で、次のような課題を解決するために作られました。

レスポンスのフォーマットが一貫しない問題:
- Function Callingが導入される前は、LLMからの返答は構造化されておらず、一貫性もありませんでした。開発者は、出力のバリエーションごとに複雑なバリデーションコードを書く必要がありました。

外部データとの連携が難しい問題:
- この機能がなかった頃は、アプリケーションの他の部分からデータをチャットの文脈に組み込むのが困難でした。

Function Callingによって、レスポンスのフォーマットが標準化され、外部データとの連携もスムーズになり、開発がシンプルになって追加のバリデーションロジックも減らせます。

たとえば「ストックホルムの現在の天気は？」のような質問には、ユーザーは答えを得られませんでした。これは、モデルが学習した時点までのデータしか扱えなかったためです。

この問題を示す例を見てみましょう。

たとえば、学生データのデータベースを作成して、最適なコースを提案したいとします。下記には、含まれているデータが非常に似ている2人の学生の説明があります。


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finshing his studies."

このデータを解析するためにLLMに送信したいと考えています。これを後でアプリケーションでAPIに送信したり、データベースに保存したりするのに利用できます。

私たちが関心のある情報についてLLMに指示する、まったく同じプロンプトを2つ作成しましょう。


私たちは、これをLLMに送信して、私たちの製品にとって重要な部分を解析したいと考えています。したがって、LLMに指示するために、2つの同一のプロンプトを作成できます。


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


これら二つのプロンプトを作成した後、`openai.ChatCompletion`を使ってLLMに送信します。プロンプトは`messages`変数に保存し、役割を`user`に割り当てます。これは、ユーザーからチャットボットへのメッセージが書き込まれることを模倣するためです。


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

プロンプトが同じで説明も似ていても、`Grades` プロパティのフォーマットは異なる場合があります。

上記のセルを何度か実行すると、フォーマットが `3.7` だったり `3.7 GPA` だったりすることがあります。

これは、LLMがプロンプトという非構造化データを受け取り、同じく非構造化データを返すためです。データを保存したり利用したりする際に予想できるよう、構造化されたフォーマットが必要です。

ファンクショナルコーリングを使うことで、構造化されたデータを受け取れるようにできます。ファンクショナルコーリングを使っても、LLMが実際に関数を呼び出したり実行したりするわけではありません。代わりに、LLMが返すレスポンスのための構造をこちらで用意します。その構造化されたレスポンスを使って、アプリケーションでどの関数を実行するか判断できるようになります。


![関数呼び出しフローダイアグラム](../../../../translated_images/Function-Flow.083875364af4f4bb69bd6f6ed94096a836453183a71cf22388f50310ad6404de.ja.png)


### 関数呼び出しの利用ケース

**外部ツールの呼び出し**  
チャットボットは、ユーザーからの質問に答えるのが得意です。関数呼び出しを使うことで、チャットボットはユーザーのメッセージをもとに特定の作業を実行できます。例えば、学生が「先生に、この科目でもっとサポートが必要だとメールして」と頼むと、`send_email(to: string, body: string)` という関数を呼び出してメールを送信できます。

**APIやデータベースクエリの作成**  
ユーザーは自然な言葉で情報を探し、それがフォーマットされたクエリやAPIリクエストに変換されます。例えば、先生が「最後の課題を終えた生徒は誰ですか」と尋ねると、`get_completed(student_name: string, assignment: int, current_status: string)` という関数を呼び出して情報を取得できます。

**構造化データの作成**  
ユーザーはテキストやCSVデータを使って、LLMで重要な情報を抽出できます。例えば、学生が平和協定に関するWikipediaの記事をAIフラッシュカードに変換したい場合、`get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)` という関数を使って、必要な事実を抽出できます。


## 2. 初めての関数呼び出しを作成する

関数呼び出しを作成する手順は、主に3つのステップに分かれています。
1. 関数のリストとユーザーからのメッセージを使って、Chat Completions APIを呼び出します
2. モデルの応答を確認し、必要なアクション（関数やAPIの呼び出しなど）を実行します
3. 関数から得られた応答を使って、再度Chat Completions APIを呼び出し、その情報をもとにユーザーへの返答を作成します


![関数呼び出しの流れ](../../../../translated_images/LLM-Flow.3285ed8caf4796d7343c02927f52c9d32df59e790f6e440568e2e951f6ffa5fd.ja.png)


### 関数呼び出しの要素

#### ユーザー入力

最初のステップは、ユーザーメッセージを作成することです。これはテキスト入力の値を動的に取得することもできますし、ここで値を指定することもできます。Chat Completions APIを初めて使う場合は、メッセージの`role`と`content`を定義する必要があります。

`role`には、`system`（ルールの作成）、`assistant`（モデル）、`user`（エンドユーザー）のいずれかを指定できます。関数呼び出しの場合は、これを`user`として、例となる質問を設定します。


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### 関数の作成

次に、関数とそのパラメータを定義します。ここでは `search_courses` という関数を一つだけ使いますが、複数の関数を作成することもできます。

**重要** : 関数はシステムメッセージに含まれ、利用可能なトークン数にも影響します。


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**定義**

関数定義の構造は複数の階層に分かれており、それぞれに固有のプロパティがあります。以下は、入れ子構造の概要です。

**最上位の関数プロパティ:**

`name` - 呼び出したい関数の名前。

`description` - 関数の動作についての説明。ここでは具体的かつ明確に記述することが重要です。

`parameters` - モデルが応答で生成する値やフォーマットの一覧。

**Parametersオブジェクトのプロパティ:**

`type` - parametersオブジェクトのデータ型（通常は "object"）

`properties` - モデルが応答で使用する具体的な値の一覧。

**個々のパラメータプロパティ:**

`name` - プロパティキーによって暗黙的に定義される（例: "role", "product", "level"）

`type` - この特定のパラメータのデータ型（例: "string", "number", "boolean"）

`description` - 特定のパラメータの説明。

**オプションのプロパティ:**

`required` - 関数呼び出しを完了するために必須となるパラメータを列挙した配列。


### 関数呼び出しの方法
関数を定義した後は、Chat Completion APIの呼び出しにその関数を含める必要があります。これには、リクエストに`functions`を追加します。今回の場合は `functions=functions` です。

さらに、`function_call` を `auto` に設定することもできます。これは、ユーザーのメッセージに応じて、どの関数を呼び出すかをLLMに任せるという意味です。自分で関数を指定するのではなく、モデルが最適な関数を選択します。


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

では、レスポンスを見て、そのフォーマットを確認しましょう。

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

このように、呼び出される関数の名前が表示されており、ユーザーのメッセージから、LLMが関数の引数に合うデータを見つけ出していることが分かります。


## 3.アプリケーションへの関数呼び出しの統合

LLMからのフォーマット済みレスポンスをテストした後、これをアプリケーションに統合できます。

### フローの管理

これをアプリケーションに統合するために、次の手順を踏みましょう。

まず、OpenAIサービスにリクエストを送り、そのメッセージを`response_message`という変数に保存します。


In [ ]:
response_message = response.choices[0].message

これから、Microsoft Learn APIを呼び出してコースのリストを取得する関数を定義します。


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



ベストプラクティスとして、まずモデルが関数を呼び出したいかどうかを確認します。その後、利用可能な関数の中から1つを作成し、呼び出されている関数に一致させます。
次に、関数の引数を取得し、LLMからの引数にマッピングします。

最後に、関数呼び出しメッセージと `search_courses` メッセージで返された値を追加します。これにより、LLMはユーザーに自然な言葉で
応答するために必要なすべての情報を得ることができます。


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## コードチャレンジ

素晴らしいですね！OpenAI Function Calling の学習をさらに深めるために、次のことに取り組んでみましょう: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst
 - 学習者がより多くのコースを見つけやすくするために、関数のパラメーターを追加してみましょう。利用可能なAPIパラメーターはこちらで確認できます:
 - 学習者の母国語など、より多くの情報を受け取る関数呼び出しを作成してみましょう
 - 関数呼び出しやAPI呼び出しで適切なコースが見つからなかった場合のエラーハンドリングを実装してみましょう



---

**免責事項**:  
本書類は、AI翻訳サービス [Co-op Translator](https://github.com/Azure/co-op-translator) を使用して翻訳されています。正確性には努めておりますが、自動翻訳には誤りや不正確な表現が含まれる場合があります。原文（元の言語の文書）が正式な情報源と見なされるべきです。重要な情報については、専門の人間による翻訳を推奨します。本翻訳の利用により生じたいかなる誤解や誤訳についても、当方は一切の責任を負いかねます。
